<a href="https://colab.research.google.com/github/Masterjun12/Generative_model/blob/main/lane_ae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="tC8ti52DSK7wGjAdfw1p")
project = rf.workspace("smt-indira-gandhi-college-of-engineering-iojgu").project("ld-le5n7")
version = project.version(4)
dataset = version.download("yolov5-obb")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ld-4 in yolov5-obb:: 100%|██████████| 1650/1650 [00:00<00:00, 7120.30it/s]


In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import shutil

source_dir = "/content/gdrive/MyDrive/notebooks/tests"
target_dir = "/content/tests"

shutil.copytree(source_dir, target_dir)


'/content/tests'

In [ ]:
import os
from PIL import Image

def parse_segmentation_info(txt_file_path):
    """
    Parses the segmentation information from the text file.
    Assumes the segmentation info is in a specific format.
    You might need to adjust this function based on your actual data format.
    """
    segmentation_info = []
    with open(txt_file_path, 'r') as file:
        for line in file:
            # Splitting by space assuming the format is like: x1 y1 x2 y2 class
            # Parsing only the first 8 elements as coordinates (ignoring class)
            coordinates = [float(x) for x in line.strip().split()[:8]]
            segmentation_info.append(coordinates)
    return segmentation_info

def crop_and_save(image_path, segmentation_info, output_folder):
    """
    Crops the image based on the segmentation information and saves it to the output folder.
    """
    image = Image.open(image_path)
    image_name = os.path.splitext(os.path.basename(image_path))[0]

    for i, seg_info in enumerate(segmentation_info):
        # Extracting coordinates
        x1, y1, x2, y2, x3, y3, x4, y4 = seg_info
        # Finding minimum and maximum coordinates for the bounding box
        min_x = min(x1, x2, x3, x4)
        min_y = min(y1, y2, y3, y4)
        max_x = max(x1, x2, x3, x4)
        max_y = max(y1, y2, y3, y4)
        # Cropping the region from the image
        cropped_region = image.crop((min_x, min_y, max_x, max_y))
        # Saving the cropped region to the output folder
        cropped_region.save(os.path.join(output_folder, f"{image_name}_crop_{i}.png"))

def process_dataset(image_folder, label_folder, output_folder):
    """
    Processes the dataset by reading images and their corresponding segmentation info,
    cropping the images based on the segmentation info, and saving the cropped images
    to the output folder.
    """
    # Ensure the output folder exists, create if not
    os.makedirs(output_folder, exist_ok=True)

    # List all image files in the image folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')]

    for image_file in image_files:
        # Get the corresponding text file
        txt_file = os.path.join(label_folder, image_file.replace('.jpg', '.txt').replace('.png', '.txt'))
        if os.path.exists(txt_file):
            # Parse segmentation info from the text file
            segmentation_info = parse_segmentation_info(txt_file)
            # Crop and save the image based on segmentation info
            crop_and_save(os.path.join(image_folder, image_file), segmentation_info, output_folder)

# Example usage:
image_folder = "/content/gdrive/MyDrive/notebooks/train/images"
label_folder = "/content/gdrive/MyDrive/notebooks/train/labelTxt"
output_folder = "/content/gdrive/MyDrive/notebooks/crop"

process_dataset(image_folder, label_folder, output_folder)


In [ ]:
import os
from PIL import Image
import numpy as np
import cv2
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

def split_train_test_images(input_folder, train_folder, test_folder, test_ratio=0.2):
    # Ensure the output folders exist, create if not
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)

    # List all image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    # Shuffle the image files
    random.shuffle(image_files)

    # Calculate the number of images for the test set
    num_test_images = int(len(image_files) * test_ratio)

    # Split the image files into train and test sets
    train_images = image_files[:-num_test_images]
    test_images = image_files[-num_test_images:]

    # Copy train images to the train folder
    for image in train_images:
        shutil.copyfile(os.path.join(input_folder, image), os.path.join(train_folder, image))

    # Copy test images to the test folder
    for image in test_images:
        shutil.copyfile(os.path.join(input_folder, image), os.path.join(test_folder, image))

# Paths
input_folder = "/content/gdrive/MyDrive/notebooks/crop"
train_folder = "/content/gdrive/MyDrive/notebooks/trains"
test_folder = "/content/gdrive/MyDrive/notebooks/tests"

# Split train and test images
split_train_test_images(input_folder, train_folder, test_folder, test_ratio=0.2)


In [ ]:
import shutil
shutil.rmtree('/content/ld-4') # 경로삭제

In [ ]:

def otsu_binarization(image):
    """
    Applies OTSU binarization to the input image.
    """
    # Convert image to numpy array
    np_image = np.array(image)
    # Convert image to grayscale
    grayscale_image = cv2.cvtColor(np_image, cv2.COLOR_RGB2GRAY)
    # Apply OTSU thresholding to the grayscale image
    _, binary_image = cv2.threshold(grayscale_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return Image.fromarray(binary_image)

def process_images_in_folder(input_folder, output_folder):
    """
    Processes all PNG images in the input folder using OTSU binarization
    and saves the binarized images to the output folder.
    """
    # Ensure the output folder exists, create if not
    os.makedirs(output_folder, exist_ok=True)

    # List all PNG files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

    for image_file in image_files:
        # Open and convert the image file to RGB mode
        image_path = os.path.join(input_folder, image_file)
        image = Image.open(image_path).convert('RGB')
        # Apply OTSU binarization
        binary_image = otsu_binarization(image)
        # Save the binarized image to the output folder
        output_path = os.path.join(output_folder, image_file)
        binary_image.save(output_path)

# Example usage:
input_folder = "/content/ld-4/train/train"
output_folder = "/content/ld-4/train/train_otsu"

process_images_in_folder(input_folder, output_folder)


In [ ]:
!pip uninstall opencv-python-headless -y
!pip uninstall opencv-contrib-python-headless -y
!pip install opencv-python-headless
!pip install opencv-contrib-python-headless


Found existing installation: opencv-python-headless 4.8.0.74
Uninstalling opencv-python-headless-4.8.0.74:
  Successfully uninstalled opencv-python-headless-4.8.0.74
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 31.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
roboflow 1.1.26 requires opencv-python-headless==4.8.0.74, but you have opencv-python-headless 4.9.0.80 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 27.4 MB/s eta 0:00:00


In [60]:
import os
import cv2
import numpy as np
from sklearn.metrics import f1_score, confusion_matrix
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
import shutil
from sklearn.metrics import f1_score, confusion_matrix

def create_autoencoder(input_shape):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    return autoencoder

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image.astype('uint8'), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    resized_image = cv2.resize(binary_image, (128, 128))
    normalized_image = resized_image.astype('float32') / 255.0
    final_image = np.reshape(normalized_image, (128, 128, 1))

    return final_image


# Function to train the Autoencoder
def train_autoencoder(input_folder):
    # Load and preprocess images for training
    image_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    images = np.array([preprocess_image(f) for f in image_files])

    # Train the Autoencoder

    checkpoint = ModelCheckpoint('autoencoder_weights.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    autoencoder.fit(images, images, epochs=200 , batch_size=32, shuffle=True, validation_split=0.2, callbacks=[checkpoint])

    return autoencoder


# Train the Autoencoder
input_folder = "/content/trains"
autoencoder = train_autoencoder(input_folder)

Epoch 1/200
47/50 [===========================>..] - ETA: 0s - loss: 0.0505
Epoch 1: val_loss improved from inf to 0.05219, saving model to autoencoder_weights.h5
50/50 [==============================] - 1s 11ms/step - loss: 0.0507 - val_loss: 0.0522
Epoch 2/200
48/50 [===========================>..] - ETA: 0s - loss: 0.0507
Epoch 2: val_loss did not improve from 0.05219
50/50 [==============================] - 0s 8ms/step - loss: 0.0508 - val_loss: 0.0523
Epoch 3/200
49/50 [============================>.] - ETA: 0s - loss: 0.0508
Epoch 3: val_loss did not improve from 0.05219
50/50 [==============================] - 0s 8ms/step - loss: 0.0508 - val_loss: 0.0522
Epoch 4/200
49/50 [============================>.] - ETA: 0s - loss: 0.0511
Epoch 4: val_loss did not improve from 0.05219
50/50 [==============================] - 0s 8ms/step - loss: 0.0512 - val_loss: 0.0524
Epoch 5/200
49/50 [============================>.] - ETA: 0s - loss: 0.0508
Epoch 5: val_loss improved from 0.05219 to 

In [62]:
# Function to detect anomalies
def detect_anomalies(autoencoder, image_path, threshold=0.01):
    # Preprocess input image
    input_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Convert to uint8
    input_image = input_image.astype('uint8')
    # Perform OTSU thresholding
    _, binary_image = cv2.threshold(input_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    resized_image = cv2.resize(binary_image, (128, 128))
    normalized_image = resized_image.astype('float32') / 255.0
    final_image = np.reshape(normalized_image, (1, 128, 128, 1))

    # Generate reconstruction from Autoencoder
    reconstructed_image = autoencoder.predict(final_image)
    # Calculate Mean Squared Error (MSE) between input and reconstructed images
    mse = np.mean(np.square(final_image - reconstructed_image))

    # Determine anomaly based on MSE and threshold
    if mse > threshold:
        return 1  # Anomaly
    else:
        return 0  # Normal


# Function to evaluate anomalies in a test folder
def evaluate_anomalies(autoencoder, test_folder):
    labels = []
    predictions = []

    for subdir in os.listdir(test_folder):
        subdir_path = os.path.join(test_folder, subdir)
        if not os.path.isdir(subdir_path):
            continue

        for image_file in os.listdir(subdir_path):
            image_path = os.path.join(subdir_path, image_file)
            label = int(subdir)  # Use folder name as label
            labels.append(label)
            prediction = detect_anomalies(autoencoder, image_path)
            predictions.append(prediction)

    return np.array(labels), np.array(predictions)

# Evaluate anomalies in the test folder
test_folder = "/content/tests"
true_labels, predicted_labels = evaluate_anomalies(autoencoder, test_folder)

# Calculate F1 score and confusion matrix
f1 = f1_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

1/1 [==============================] - 0s 23ms/step
F1 Score: 0.163265306122449
Confusion Matrix:
[[445  12]
 [ 29   4]]
